# Binary classifier for built-up areas

This notebook contains useful code snippets one can use if they want to replicate the procedure described in the Medium [blog post](https://medium.com/p/7f2d7114ed1c/) that showed how to train a binary classifier to identify built-up areas.

### LineString geometry handling

Roads in OSM data are represented as LineString geometries. These can range from short to several kilometers long. We cut these down to 100 meter lengths so they are more manageble. This can be done with the code below:

In [3]:
from typing import List, Union

from shapely.geometry import LineString, Point

def cut(line: LineString, distance: Union[int, float], lines: List[LineString]):
    """ 
    Cuts the provided 'line' in several segments at a 'distance' from its starting point.
    The cut lines are appended to an existing List of LineString geometries.
    """
    
    if distance <= 0.0 or distance >= line.length:
        return [LineString(line)]
    
    coords = list(line.coords)
    for i, p in enumerate(coords):
        pd = line.project(Point(p))
        if pd == distance:
            return [
                LineString(coords[:i+1]),
                LineString(coords[i:])
            ]
        if pd > distance:
            cp = line.interpolate(distance)
            lines.append(LineString(coords[:i] + [(cp.x, cp.y)]))
            line = LineString([(cp.x, cp.y)] + coords[i:])
            if line.length > distance:
                cut(line, distance, lines)
            else:
                lines.append(LineString([(cp.x, cp.y)] + coords[i:]))
            return lines

Things to note. For the `distance` one would like to cut the line to be in the units of meters, the lines have to be in the corresponding UTM CRS.

## Parsing the LightGBM model

The code below parses the decision trees which are part of the LightGBM model into string representations held insight a Javascript file. This can be used to later visualize large areas in [EO Browser](https://apps.sentinel-hub.com/eo-browser/).

In [4]:
import joblib

### defining inputs and stringifying parameters

PRECISION_SCORES = 4
PRECISION_THRESHOLD = None

MODEL_INPUTS = [] #specify your model inputs

BANDS = ['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B09', 'B11', 'B12']

BANDS_STR = ','.join(BANDS)
MODEL_INPUTS_STR = ', '.join(MODEL_INPUTS)

### defining model parsing functions

def parse_subtree(node,  model_inputs, brackets=True):
    if 'leaf_index' in node:
        score = float(node["leaf_value"])
        if PRECISION_SCORES is not None:
            score = round(score, PRECISION_SCORES)
        return f'{score}'
    
    feature = model_inputs[int(node["split_feature"])]
    
    threshold = float(node["threshold"])
    if PRECISION_THRESHOLD is not None:
        threshold = round(threshold, PRECISION_THRESHOLD)
    
    condition = f'{feature}{node["decision_type"]}{threshold}'
    
    left = parse_subtree(node['left_child'], model_inputs)
    right = parse_subtree(node['right_child'], model_inputs)
    
    result = f'({condition})?{left}:{right}'
    if brackets:
        return f'({result})'
    return result


def parse_one_tree(root, index, model_input_str, model_inputs):
    return \
f"""
function pt{index}({model_input_str}) {{ 
   return {parse_subtree(root, model_inputs, brackets=False)};
}}
"""


def parse_trees(trees):
    
    tree_functions = '\n'.join([parse_one_tree(tree['tree_structure'], idx, MODEL_INPUTS_STR, MODEL_INPUTS)
                                  for idx, tree in enumerate(trees)])
    function_sum = '+'.join([f'pt{i}({MODEL_INPUTS_STR})' for i in range(len(trees))])
    
    return f"""
//VERSION=3
function setup() {{
    return {{
        input: [{{
            bands: [{','.join(f'"{band}"' for band in BANDS)}],
            units: "DN"
        }}],
        output: {{
            id:"default",
            bands: 4,
            sampleType: "AUTO"
        }}
    }}
}}
function evaluatePixel(sample) {{
    // define the indices you need
    let NDVI = (sample.B08 - sample.B04)/(sample.B08 + sample.B04)
    let NDVI_RE1 = (sample.B08 - sample.B05)/(sample.B08 + sample.B05)
    
    ...
    
    let STI = sample.B11 / sample.B12
    
    // call the model. Keep in mind that DN have to be divided by 1.e4 to get the same values as the models so when training
    let BLDG_PROBA = predict(sample.B01/1.e4, sample.B02/1.e4,..., STI)
    
    // make a mask of predictions
    let BLDG_PROBA_080 = (BLDG_PROBA > 0.8) ? 1 : 0;
    
    // return a blue mask where the mask is used as a transparency layer. With minor modificaitons one can also return the probabilities themselves.
    return [0, 0, 0, 0.4 * BLDG_PROBA_080]
}}
{tree_functions}
function predict({MODEL_INPUTS_STR}) {{ 
    return [1/(1+Math.exp(-1*({function_sum})))];
}}
"""

def parse_model(model, js_output_filename=None):
    model_json = model.booster_.dump_model()

    model_javascript = parse_trees(model_json['tree_info'])
    
    if js_output_filename:
        with open(js_output_filename, 'w') as f:
            f.write(model_javascript)
        
    return model_javascript